In [20]:
import pandas as pd
import numpy as np


In [21]:

TABLE_FOLDER = '../../src/data'
RAW_FILE = 'games_2000_2022.csv'


# Introduction
This Notebooks reads the raw data created with the scrapping and creates new features.


In [22]:

data = pd.read_csv(f'{TABLE_FOLDER}/{RAW_FILE}', parse_dates=['date'], dayfirst=True)
u = [u for u in data.columns.tolist() if 'Unnamed' in u]
data.drop(u, axis=1, inplace=True)
print(data.shape)
data.sample(8)

(8353, 10)


,date,home_team,away_team,score,odd_1,odd_N,odd_2,winner,league_season,game_url
5337,2015-05-16,Sunderland,Leicester,0:0,2.94,3.23,2.54,1.0,premier-league-2014-2015,/soccer/england/premier-league-2014-2015/sunde...
2814,2008-01-19,Blackburn,Middlesbrough,1:1,2.01,3.39,4.38,1.0,premier-league-2007-2008,/soccer/england/premier-league-2007-2008/black...
4073,2010-11-06,Blackburn,Wigan,2:1,1.72,3.60,4.94,0.0,premier-league-2010-2011,/soccer/england/premier-league-2010-2011/black...
4715,2013-01-19,Newcastle,Reading,1:2,1.66,3.83,5.37,2.0,premier-league-2012-2013,/soccer/england/premier-league-2012-2013/newca...
1109,2002-08-31,Sunderland,Manchester Utd,1:1,5.08,3.53,1.69,1.0,premier-league-2002-2003,/soccer/england/premier-league-2002-2003/sunde...
5819,2016-02-14,Aston Villa,Liverpool,0:6,5.46,3.75,1.70,2.0,premier-league-2015-2016,/soccer/england/premier-league-2015-2016/aston...
5737,2016-04-23,Bournemouth,Chelsea,1:4,3.39,3.55,2.16,2.0,premier-league-2015-2016,/soccer/england/premier-league-2015-2016/bourn...
6877,2019-04-16,Brighton,Cardiff,0:2,2.00,3.25,4.37,2.0,premier-league-2018-2019,/soccer/england/premier-league-2018-2019/brigh...


In [23]:
data.dtypes

date             datetime64[ns]
home_team                object
away_team                object
score                    object
odd_1                   float64
odd_N                   float64
odd_2                   float64
winner                  float64
league_season            object
game_url                 object
dtype: object

## Last x games result
The scope is to create 2 fields, one for the home_team and another one for the away_team, containing the last x games with
- W won
- D draw
- L lost

In [24]:
# create a list of all teams in the data
ht_list = np.unique(data.home_team.tolist()).tolist()
at_list = np.unique(data.away_team.tolist()).tolist()
all_teams = np.unique([at_list + ht_list])


In [25]:
print(data.shape)
data = data.dropna()
data.shape


(8353, 10)


(8347, 10)

In [26]:

for team in all_teams:
    #team = 'Arsenal'
    temp=data.query('home_team == @team or away_team == @team').sort_values('date')
    past5 = ''
    for idx, r in temp.iterrows():
        scores = str(r.score).split(':')
        
        if r.away_team == team:
            temp.loc[idx,'h_score']=scores[0]
            temp.loc[idx,'a_score']=scores[1]

            temp.loc[idx,'h-_score']=scores[1]
            temp.loc[idx,'a-_score']=scores[0]
            
            
            if r.winner ==  2:
                temp.loc[idx,'past5']='Wa'
            elif r.winner == 1:
                temp.loc[idx,'past5']='Da'
            else:
                temp.loc[idx,'past5']='La'
        else:
            temp.loc[idx,'h_score']=scores[1]
            temp.loc[idx,'a_score']=scores[0]

            temp.loc[idx,'h-_score']=scores[0]
            temp.loc[idx,'a-_score']=scores[1]

        
            if r.winner ==  0:
                temp.loc[idx,'past5']='Wh'
            elif r.winner == 1:
                temp.loc[idx,'past5']='Dh'
            else:
                temp.loc[idx,'past5']='Lh'
    temp['past5_1'] = temp['past5'].shift(1)
    temp['past5_2'] = temp['past5'].shift(2)
    temp['past5_3'] = temp['past5'].shift(3)
    temp['past5_4'] = temp['past5'].shift(4)
    temp['past5_5'] = temp['past5'].shift(5)       
    
    temp['h5_1'] = temp['h_score'].shift(1) 
    temp['h5_2'] = temp['h_score'].shift(2) 
    temp['h5_3'] = temp['h_score'].shift(3) 
    temp['h5_4'] = temp['h_score'].shift(4) 
    temp['h5_5'] = temp['h_score'].shift(5) 
    
    temp['wh5_1'] = temp['h-_score'].shift(1) 
    temp['wh5_2'] = temp['h-_score'].shift(2) 
    temp['wh5_3'] = temp['h-_score'].shift(3) 
    temp['wh5_4'] = temp['h-_score'].shift(4) 
    temp['wh5_5'] = temp['h-_score'].shift(5) 
    
    

    temp['a5_1'] = temp['a_score'].shift(1) 
    temp['a5_2'] = temp['a_score'].shift(2) 
    temp['a5_3'] = temp['a_score'].shift(3) 
    temp['a5_4'] = temp['a_score'].shift(4) 
    temp['a5_5'] = temp['a_score'].shift(5) 

    temp['wa5_1'] = temp['a-_score'].shift(1) 
    temp['wa5_2'] = temp['a-_score'].shift(2) 
    temp['wa5_3'] = temp['a-_score'].shift(3) 
    temp['wa5_4'] = temp['a-_score'].shift(4) 
    temp['wa5_5'] = temp['a-_score'].shift(5) 

    # golal scored
    





    temp=temp[['date','past5_1','past5_2','past5_3','past5_4','past5_5',
               'h5_1','h5_2','h5_3','h5_4','h5_5',
               'a5_1','a5_2','a5_3','a5_4','a5_5',
                'wh5_1','wh5_2','wh5_3','wh5_4','wh5_5',
               'wa5_1','wa5_2','wa5_3','wa5_4','wa5_5',           
               ]]
    temp['team']=team
    temp

    for idx, d in data.query('home_team == @team').iterrows():
        # get last game from temp
        gameday = d.date
        v=temp.query('date == @gameday and team == @team')
        data.loc[idx,'h_past5_1']=v.past5_1.values[0]
        data.loc[idx,'h_past5_2']=v.past5_2.values[0]
        data.loc[idx,'h_past5_3']=v.past5_3.values[0]
        data.loc[idx,'h_past5_4']=v.past5_4.values[0]
        data.loc[idx,'h_past5_5']=v.past5_5.values[0]
        
        data.loc[idx,'h_score_1']=v.h5_1.values[0]
        data.loc[idx,'h_score_2']=v.h5_2.values[0]
        data.loc[idx,'h_score_3']=v.h5_3.values[0]
        data.loc[idx,'h_score_4']=v.h5_4.values[0]
        data.loc[idx,'h_score_5']=v.h5_5.values[0]

        data.loc[idx,'wh_score_1']=v.wh5_1.values[0]
        data.loc[idx,'wh_score_2']=v.wh5_2.values[0]
        data.loc[idx,'wh_score_3']=v.wh5_3.values[0]
        data.loc[idx,'wh_score_4']=v.wh5_4.values[0]
        data.loc[idx,'wh_score_5']=v.wh5_5.values[0]
        
        
        
    for idx, d in data.query('away_team == @team').iterrows():
        # get last game from temp
        gameday = d.date
        v=temp.query('date == @gameday and team == @team')
        data.loc[idx,'a_past5_1']=v.past5_1.values[0]
        data.loc[idx,'a_past5_2']=v.past5_2.values[0]
        data.loc[idx,'a_past5_3']=v.past5_3.values[0]
        data.loc[idx,'a_past5_4']=v.past5_4.values[0]
        data.loc[idx,'a_past5_5']=v.past5_5.values[0]
        
        data.loc[idx,'a_score_1']=v.a5_1.values[0]
        data.loc[idx,'a_score_2']=v.a5_2.values[0]
        data.loc[idx,'a_score_3']=v.a5_3.values[0]
        data.loc[idx,'a_score_4']=v.a5_4.values[0]
        data.loc[idx,'a_score_5']=v.a5_5.values[0]
         
        data.loc[idx,'wa_score_1']=v.wa5_1.values[0]
        data.loc[idx,'wa_score_2']=v.wa5_2.values[0]
        data.loc[idx,'wa_score_3']=v.wa5_3.values[0]
        data.loc[idx,'wa_score_4']=v.wa5_4.values[0]
        data.loc[idx,'wa_score_5']=v.wa5_5.values[0]
       
        
        

In [27]:
past = [p for p in data.columns.tolist() if '_past5' in p]
data.dropna(inplace=True)
data

,date,home_team,away_team,score,odd_1,odd_N,odd_2,winner,league_season,game_url,...,a_score_1,a_score_2,a_score_3,a_score_4,a_score_5,wa_score_1,wa_score_2,wa_score_3,wa_score_4,wa_score_5
0,2001-05-19,Charlton,Liverpool,0:4,3.60,3.60,1.70,2.0,premier-league-2000-2001,/soccer/england/premier-league-2000-2001/charl...,...,2,3,2,2,3,2,0,0,0,1
1,2001-05-19,Coventry,Bradford City,0:0,1.79,3.30,3.80,1.0,premier-league-2000-2001,/soccer/england/premier-league-2000-2001/coven...,...,1,1,0,1,2,6,1,2,2,0
2,2001-05-19,Derby,Ipswich,1:1,3.50,3.30,1.96,1.0,premier-league-2000-2001,/soccer/england/premier-league-2000-2001/derby...,...,2,1,2,2,1,1,2,0,1,0
3,2001-05-19,Everton,Sunderland,2:2,2.50,3.10,2.50,1.0,premier-league-2000-2001,/soccer/england/premier-league-2000-2001/evert...,...,3,1,1,0,2,2,0,1,1,3
4,2001-05-19,Leeds,Leicester,3:1,1.27,4.50,9.99,0.0,premier-league-2000-2001,/soccer/england/premier-league-2000-2001/leeds...,...,4,0,0,0,1,2,1,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8347,2021-08-14,Everton,Southampton,3:1,2.04,3.38,3.95,0.0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/evert...,...,0,0,3,3,0,3,2,1,1,2
8348,2021-08-14,Leicester,Wolves,1:0,1.74,3.73,5.06,0.0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/leice...,...,1,0,0,2,1,2,1,2,1,1
8349,2021-08-14,Manchester Utd,Leeds,5:1,1.64,4.19,5.21,0.0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/manch...,...,3,2,4,3,0,1,0,0,1,2
8350,2021-08-14,Norwich,Liverpool,0:3,6.75,4.85,1.46,2.0,premier-league-2021-2022,/soccer/england/premier-league-2021-2022/norwi...,...,2,3,2,4,2,0,0,1,2,0


In [28]:
data.columns

Index(['date', 'home_team', 'away_team', 'score', 'odd_1', 'odd_N', 'odd_2',
       'winner', 'league_season', 'game_url', 'h_past5_1', 'h_past5_2',
       'h_past5_3', 'h_past5_4', 'h_past5_5', 'h_score_1', 'h_score_2',
       'h_score_3', 'h_score_4', 'h_score_5', 'wh_score_1', 'wh_score_2',
       'wh_score_3', 'wh_score_4', 'wh_score_5', 'a_past5_1', 'a_past5_2',
       'a_past5_3', 'a_past5_4', 'a_past5_5', 'a_score_1', 'a_score_2',
       'a_score_3', 'a_score_4', 'a_score_5', 'wa_score_1', 'wa_score_2',
       'wa_score_3', 'wa_score_4', 'wa_score_5'],
      dtype='object')

In [29]:
team1='Crystal Palace'
team2='Arsenal'
s = [s for s in data.columns.tolist() if 'score' in s]
s = ['home_team','away_team','winner','odd_1','odd_N','odd_2'] + s
data.query('home_team == @team1 and away_team == @team2')[s]

,home_team,away_team,winner,odd_1,odd_N,odd_2,score,h_score_1,h_score_2,h_score_3,...,a_score_1,a_score_2,a_score_3,a_score_4,a_score_5,wa_score_1,wa_score_2,wa_score_3,wa_score_4,wa_score_5
1786,Crystal Palace,Arsenal,1.0,6.85,3.96,1.41,1:1,0,0,1,...,2,0,3,4,1,2,2,1,0,0
5234,Crystal Palace,Arsenal,2.0,11.46,5.50,1.29,0:2,4,3,2,...,4,1,2,3,3,1,1,1,1,1
5444,Crystal Palace,Arsenal,2.0,4.83,3.68,1.77,1:2,1,0,1,...,2,1,5,2,3,1,2,0,0,0
6055,Crystal Palace,Arsenal,2.0,5.09,3.93,1.70,1:2,0,1,2,...,0,4,0,1,0,2,1,0,1,1
6140,Crystal Palace,Arsenal,0.0,4.62,4.10,1.74,3:0,3,1,0,...,3,2,1,1,2,0,2,3,3,0
6633,Crystal Palace,Arsenal,2.0,4.57,4.17,1.74,2:3,1,0,1,...,3,1,0,1,1,3,0,0,1,3
7115,Crystal Palace,Arsenal,1.0,4.50,3.92,1.79,2:2,2,1,2,...,3,5,2,2,2,1,1,0,0,1
7377,Crystal Palace,Arsenal,1.0,4.58,3.88,1.77,1:1,1,1,1,...,2,1,1,0,0,0,2,1,0,3
7604,Crystal Palace,Arsenal,2.0,5.91,4.12,1.59,1:3,2,3,0,...,1,3,2,0,1,0,1,0,1,1
8055,Crystal Palace,Arsenal,0.0,4.03,3.38,2.01,3:0,0,0,1,...,1,0,2,3,2,0,2,0,2,1


In [30]:
team1='Crystal Palace'
team2='Arsenal'
s = [s for s in data.columns.tolist() if 'past' in s]
s = ['date','home_team','away_team','winner','odd_1','odd_N','odd_2'] + s
data.query('home_team == @team1 and away_team == @team2')[s]

,date,home_team,away_team,winner,odd_1,odd_N,odd_2,h_past5_1,h_past5_2,h_past5_3,h_past5_4,h_past5_5,a_past5_1,a_past5_2,a_past5_3,a_past5_4,a_past5_5
1786,2004-11-06,Crystal Palace,Arsenal,1.0,6.85,3.96,1.41,Wa,Wh,La,Wh,Da,Dh,La,Wh,Wh,Wa
5234,2013-10-26,Crystal Palace,Arsenal,2.0,11.46,5.50,1.29,Lh,La,La,Lh,La,Wh,Da,Wa,Wh,Wa
5444,2015-02-21,Crystal Palace,Arsenal,2.0,4.83,3.68,1.77,Dh,Wa,Lh,Wa,Wh,Wh,La,Wh,Wa,Wh
6055,2015-08-16,Crystal Palace,Arsenal,2.0,5.09,3.93,1.70,Wh,Wa,Lh,La,Lh,Lh,Wh,Dh,Da,Lh
6140,2017-04-10,Crystal Palace,Arsenal,0.0,4.62,4.10,1.74,La,Wa,Wh,Wa,Wh,Wh,Dh,La,La,Wh
6633,2017-12-28,Crystal Palace,Arsenal,2.0,4.57,4.17,1.74,Da,Wa,Wh,Dh,Da,Dh,Wh,Da,Da,Lh
7115,2018-10-28,Crystal Palace,Arsenal,1.0,4.50,3.92,1.79,La,Lh,La,Dh,Wa,Wh,Wa,Wh,Wh,Wa
7377,2020-01-11,Crystal Palace,Arsenal,1.0,4.58,3.88,1.77,Da,Da,Wh,La,Dh,Wh,Lh,Da,Da,Lh
7604,2021-05-19,Crystal Palace,Arsenal,2.0,5.91,4.12,1.59,Wh,La,Wa,Lh,La,Wa,Wh,Wa,Lh,Dh
8055,2022-04-04,Crystal Palace,Arsenal,0.0,4.03,3.38,2.01,Dh,Wa,Dh,Wa,Lh,Wa,Lh,Wh,Wa,Wh


In [31]:
data.shape

(8179, 40)

In [32]:
data.columns

Index(['date', 'home_team', 'away_team', 'score', 'odd_1', 'odd_N', 'odd_2',
       'winner', 'league_season', 'game_url', 'h_past5_1', 'h_past5_2',
       'h_past5_3', 'h_past5_4', 'h_past5_5', 'h_score_1', 'h_score_2',
       'h_score_3', 'h_score_4', 'h_score_5', 'wh_score_1', 'wh_score_2',
       'wh_score_3', 'wh_score_4', 'wh_score_5', 'a_past5_1', 'a_past5_2',
       'a_past5_3', 'a_past5_4', 'a_past5_5', 'a_score_1', 'a_score_2',
       'a_score_3', 'a_score_4', 'a_score_5', 'wa_score_1', 'wa_score_2',
       'wa_score_3', 'wa_score_4', 'wa_score_5'],
      dtype='object')

In [33]:
for idx, r in data.iterrows():
    data.loc[idx,'last_5_home']= f'{r.h_past5_1}/{r.h_past5_2}/{r.h_past5_3}/{r.h_past5_4}/{r.h_past5_5}' 
    data.loc[idx,'last_5_away']= f'{r.a_past5_1}/{r.a_past5_2}/{r.a_past5_3}/{r.a_past5_4}/{r.a_past5_5}' 
    data.loc[idx,'last_h_goals'] = int(r.h_score_1) + int(r.h_score_2) + int(r.h_score_3) + int(r.h_score_4) + int(r.h_score_5) +0
    data.loc[idx,'last_a_goals'] = int(r.a_score_1) + int(r.a_score_2) +int(r.a_score_3) + int(r.a_score_4) + int(r.a_score_5) +0
    data.loc[idx,'last_wh_goals'] = int(r.wh_score_1) + int(r.wh_score_2) + int(r.wh_score_3) + int(r.wh_score_4) + int(r.wh_score_5) +0
    data.loc[idx,'last_wa_goals'] = int(r.wa_score_1) + int(r.wa_score_2) + int(r.wa_score_3) + int(r.wa_score_4) + int(r.wa_score_5) +0
   
    
    

In [34]:
s = [s for s in data.columns.tolist() if 'last' in s]
s = s + ['odd_1','odd_N','odd_2','winner']
print(s)
data[s]

['last_5_home', 'last_5_away', 'last_h_goals', 'last_a_goals', 'last_wh_goals', 'last_wa_goals', 'odd_1', 'odd_N', 'odd_2', 'winner']


,last_5_home,last_5_away,last_h_goals,last_a_goals,last_wh_goals,last_wa_goals,odd_1,odd_N,odd_2,winner
0,LaWhWaDhLa,DhWhWaWaWh,9.0,12.0,8.0,3.0,3.60,3.60,1.70,2.0
1,LaLhLaWhLa,LaDhLhLaWh,11.0,5.0,5.0,11.0,1.79,3.30,3.80,1.0
2,WaLhLaWhLa,WhLaWhWaWh,7.0,8.0,5.0,4.0,3.50,3.30,1.96,1.0
3,LaWhLaLhLa,WhWaDhLaLh,12.0,7.0,7.0,7.0,2.50,3.10,2.50,1.0
4,WhLaWhWaWa,WhLaLhLaLh,4.0,5.0,13.0,10.0,1.27,4.50,9.99,0.0
...,...,...,...,...,...,...,...,...,...,...
8347,LaWhLhDaWa,LaLhWhWhLa,6.0,6.0,2.0,9.0,2.04,3.38,3.95,0.0
8348,LhLaWaLhDa,LhLaLaWhDa,12.0,4.0,8.0,7.0,1.74,3.73,5.06,0.0
8349,WaDhLhLhWa,WhWaWaWhLa,9.0,12.0,9.0,4.0,1.64,4.19,5.21,0.0
8350,LaLhLaLhLa,WhWaWaWaWh,14.0,13.0,1.0,3.0,6.75,4.85,1.46,2.0


In [35]:

TABLE_FOLDER = '../../src/data'
RAW_FILE = 'modeling_v1_data.csv'

data[s].to_csv(f'{TABLE_FOLDER}/{RAW_FILE}')

In [36]:
data[s].dtypes

last_5_home       object
last_5_away       object
last_h_goals     float64
last_a_goals     float64
last_wh_goals    float64
last_wa_goals    float64
odd_1            float64
odd_N            float64
odd_2            float64
winner           float64
dtype: object

In [37]:
data[s].drop_duplicates()

,last_5_home,last_5_away,last_h_goals,last_a_goals,last_wh_goals,last_wa_goals,odd_1,odd_N,odd_2,winner
0,LaWhWaDhLa,DhWhWaWaWh,9.0,12.0,8.0,3.0,3.60,3.60,1.70,2.0
1,LaLhLaWhLa,LaDhLhLaWh,11.0,5.0,5.0,11.0,1.79,3.30,3.80,1.0
2,WaLhLaWhLa,WhLaWhWaWh,7.0,8.0,5.0,4.0,3.50,3.30,1.96,1.0
3,LaWhLaLhLa,WhWaDhLaLh,12.0,7.0,7.0,7.0,2.50,3.10,2.50,1.0
4,WhLaWhWaWa,WhLaLhLaLh,4.0,5.0,13.0,10.0,1.27,4.50,9.99,0.0
...,...,...,...,...,...,...,...,...,...,...
8347,LaWhLhDaWa,LaLhWhWhLa,6.0,6.0,2.0,9.0,2.04,3.38,3.95,0.0
8348,LhLaWaLhDa,LhLaLaWhDa,12.0,4.0,8.0,7.0,1.74,3.73,5.06,0.0
8349,WaDhLhLhWa,WhWaWaWhLa,9.0,12.0,9.0,4.0,1.64,4.19,5.21,0.0
8350,LaLhLaLhLa,WhWaWaWaWh,14.0,13.0,1.0,3.0,6.75,4.85,1.46,2.0
